In [2]:
# Directories
import os
os.chdir(r"E:\academy\OneDrive - IIT Delhi\3. IIT DELHI\2. Academics\1_PAPER\2_Analysis")
print(os.getcwd())

E:\academy\OneDrive - IIT Delhi\3. IIT DELHI\2. Academics\1_PAPER\2_Analysis


In [3]:
#Array packages
import cartopy.crs as ccrs
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nc4

from scipy.stats import kendalltau
import pymannkendall as mk

#plots
import matplotlib.pyplot as plt
import rioxarray as rio
import geopandas as gpd
from shapely.geometry import mapping
import matplotlib.ticker as ticker

#Progress meter
from dask.diagnostics import ProgressBar
from tqdm import tqdm

# Directories
import os
import glob
import dask
#import h5netcdf
import scipy


C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
mdl=5

## 1 Full timeseries

Lis outpt conversion
Lis output - kg/m2s = lis *60*60*24 mm/day

#### 1.1 Grid

In [7]:

model=["LIS_MERRA2","LIS_IMD","LIS_CHIRPS","LIS_ERA5","LIS_IMDc","LIS_IMDcnew"]
var=['TotalPrecip_tavg','Evap_tavg','Qs_tavg','Qsb_tavg']

lsm = xr.open_mfdataset(f"DATA_p/{model[mdl]}_1980_2021.nc",engine='netcdf4', parallel=True,chunks={'time': 'auto'})
lsm=lsm[var]
#lsm=lsm.drop_vars('spatial_ref')
lsm=lsm*60*60*24*30  # This 30 comes because we accidently averaged the daily to instead of doingg accumulated sum in HPC
lsm=lsm.where(lsm>=0) # removin g negatives
lsm=lsm.where((lsm>=0.1)|(lsm.isnull()),0) #isnull is to make the outer location nans remains nan

lsm_re=lsm.where(lsm.TotalPrecip_tavg>=0.4)  # This only applied to calculate RE so that RE>1 can be removed
lsm_re=lsm_re.where(~(lsm_re["Qs_tavg"]>lsm_re["TotalPrecip_tavg"]))

#RE calculation
RE=lsm_re["Qs_tavg"]/lsm_re["TotalPrecip_tavg"]
RE=RE.where(~np.isinf(RE)) #converting inf to nan (inf effect all operation like mean, trend etc)
RE=RE.where(RE<0.7).compute()

lsm['RE']=RE


#or linearly interpolate
RE_fin=RE.interpolate_na(dim="lon", method="linear") # lat or lon is also fine
#RE_fin=RE_fin.interpolate_na(dim="lat", method="linear")
#RE_fin=RE_fin.interpolate_na(dim="time", method="linear")

#Attaching seasonal mean
RE_mn=RE.groupby("time.month").mean()
RE_mn = xr.concat([RE_mn]*len(np.unique(RE.time.dt.year)), dim="month")
RE_mn1=RE_mn.rename({'month': 'time'});RE_mn1["time"]=RE.time
RE_fin=RE_fin.combine_first(RE_mn1)


RE_fin=RE_fin.where(~((RE.isnull()) & (lsm['TotalPrecip_tavg'].isnull())),RE) # This is to refil back the sorround nan values

lsm['RE_cleaned']=RE_fin

#lsm=lsm.drop_vars('spatial_ref')
lsm.to_netcdf(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_grid_All.nc')

In [4]:
lsm.to_netcdf(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_grid_All.nc')

#### 1.2 Region

In [8]:
gdf_p=gpd.read_file('DATA_p/shapefile/south_asia_p.shp')
dataset=[]
lsm_R1=lsm
for i in range(len(gdf_p.Basin)):
    
    B=gdf_p[gdf_p['Basin'] == gdf_p['Basin'][i]]
    lsm_R1.rio.set_spatial_dims(y_dim="lat",x_dim="lon", inplace=True)
    lsm_R1.rio.write_crs("EPSG:4326", inplace=True)
    lsm_clip = lsm_R1.rio.clip(B.geometry.apply(mapping), B.crs, drop=True)
    A=lsm_clip.mean(dim=['lat','lon']).expand_dims(dim='region')
    dataset.append(A)
    
lsm_R_all = xr.concat(dataset, dim='region')
lsm_R_all['region'] = np.array(gdf_p.Basin)
lsm_R_all.to_netcdf(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_reg_All.nc')

# 2 SPI

#### 2.1 grid

In [10]:
import climate_indices
from climate_indices import indices
import warnings
warnings.filterwarnings("ignore")

ds=lsm[['TotalPrecip_tavg','Qs_tavg','Evap_tavg','RE','RE_cleaned']].compute()


ds=ds.where(ds!=0,0.001);ds=ds.where(ds>=0);
da_precip_groupby = ds.stack(point=('lat', 'lon')).groupby('point')
da_precip_groupby

scale = 1
distribution = climate_indices.indices.Distribution.gamma
data_start_year = 1980
calibration_year_initial = 1980
calibration_year_final = 2021
periodicity = periodicity = climate_indices.compute.Periodicity.monthly

da_spi = xr.apply_ufunc(indices.spi,
                        da_precip_groupby,
                        scale,
                        distribution,
                        data_start_year,
                        calibration_year_initial,
                        calibration_year_final,
                        periodicity)

# unstack the array back into original dimensions
da_spi = da_spi.unstack('point')

da_spi.to_netcdf(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_grid_SPI1.nc')



KeyboardInterrupt



#### 2.2 REGIONAL

In [9]:
import numpy as np
import xarray as xr
from scipy.stats import gamma, norm
def spi_cal(data,yr):
    data1 = data.copy()

    for i in range(12): 
        sq=np.arange(i, i + 12*yr, 12)
        D=data1[sq]
        D[D == 0] = 0.01
        D1=D[~np.isnan(D)]
        shape, loc, scale = gamma.fit(D1)
        POE = 1-gamma.cdf(D, shape, loc, scale)
        cdf_std = norm.cdf(norm.ppf(POE))
        spi = norm.ppf(1 - cdf_std)
        data1[sq]=spi
    return data1


ds1=lsm_R_all.compute()
SPI = xr.apply_ufunc(
    spi_cal,  # Function for wavelet transformation
    ds1,  # Input dataset
    input_core_dims=[['time']],  # Core dimensions of the input
    output_core_dims=[['time']],  # Core dimensions of the output
    vectorize=True,  # Treat each lat-lon pair as a separate input
    dask='parallelized',  # Use parallelized computation if using Dask arrays
    output_dtypes=[ds1.RE.dtype],  # Output data type
    output_sizes={'time': len(ds1.time.values)},# Size of the output dimension
    kwargs={'yr': len(np.unique(ds1['time.year']))}
)
SPI=SPI.where(~np.isinf(SPI),0)
SPI.to_netcdf(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_reg_SPI1.nc')

C:\Users\sstar\AppData\Local\Temp\ipykernel_30672\1836767655.py:21: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  SPI = xr.apply_ufunc(


In [32]:
xr.corr(lsm_R_all.TotalPrecip_tavg,spi_reg.SPI_TotalPrecip_tavg,dim='time').compute()

<xarray.DataArray (region: 18)> Size: 144B
array([0.06857724, 0.25589823, 0.5080208 , 0.74392576, 0.74248765,
       0.58761546, 0.64329049, 0.21103489, 0.47981322, 0.49971078,
       0.58909048, 0.54863124, 0.25077406, 0.37901431, 0.28986268,
       0.50130565, 0.30504611, 0.84604764])
Coordinates:
    spatial_ref  int32 4B 0
  * region       (region) object 144B 'Indus' 'Ganga' ... 'EFMP' 'West Flowing'

# 3 Seasonal

#### 3.1 Grid

In [5]:
ssn=['Pre-monsoon','Monsoon','Post-monsoon','Winter']
ssn_m=[[3,4,5],[6,7,8,9],[10,11],[12,1,2]]
model=["LIS_MERRA2","LIS_IMD","LIS_CHIRPS","LIS_ERA5","LIS_IMDc","LIS_IMDcnew"]
#var=['TotalPrecip_tavg','Evap_tavg','Qs_tavg']


lsm = xr.open_mfdataset(f"DATA_p/{model[mdl]}_1980_2021.nc",engine='netcdf4', parallel=True,chunks={'time': 'auto'})
#lsm=lsm.compute()
lsm=lsm*60*60*24*30
lsm=lsm.where(lsm>0)
lsm_re=lsm.where((lsm>=0.1)|(lsm.isnull()),0) #is

dataset=[]
for ss in range(4):
    if ss!=3:
        lsm_ssn=lsm.sel(time=lsm["time.month"].isin(ssn_m[ss]))
        lsm_ssn=lsm_ssn.groupby("time.year").mean(dim="time")
        print("1")
    else:
        lsm_roll = lsm.rolling(time=3).mean()
        lsm_ssn=lsm_roll.isel(time=lsm_roll["time.month"].isin([1])).groupby("time.year").mean()
        lsm_ssn=lsm_ssn.where(lsm_ssn['year'] != 1980, lsm_ssn.isel(year=1))
        print("3")

    lsm_re=lsm_ssn.where(lsm_ssn.TotalPrecip_tavg>=0.4) #isnull is to make the outer location nans remains nan
    lsm_re1=lsm_re.where(~(lsm_ssn["Qs_tavg"]>lsm_ssn["TotalPrecip_tavg"]))
     
    RE=lsm_re1["Qs_tavg"]/lsm_re1["TotalPrecip_tavg"]
    RE=RE.where(~np.isinf(RE)) #converting inf to nan (inf effect all operation like mean, trend etc)
    RE=RE.where(RE<0.7).compute()
    lsm_ssn['RE']=RE
    
    #Filling values 

    RE_fin=RE.interpolate_na(dim="lon", method="linear") # lat or lon is also fine
    #RE_fin=RE_fin.interpolate_na(dim="lat", method="linear")
    #RE_fin=RE_fin.interpolate_na(dim="year", method="linear")
    
    lsm_ssn['RE_cleaned']=RE_fin
    
    dataset.append(lsm_ssn.expand_dims(dim='season'))

lsm_ssn = xr.concat(dataset, dim='season')
lsm_ssn=lsm_ssn.assign_coords(season=ssn)


#lsm_ssn=lsm_ssn.drop_vars('spatial_ref')
lsm_ssn.to_netcdf(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_grid_ssn.nc')



1
1
1
3


#### 3.2 Regional

In [6]:
gdf_p=gpd.read_file('DATA_p/shapefile/south_asia_p.shp')
dataset=[]
lsm_R2=lsm_ssn
for i in range(len(gdf_p.Basin)):
    
    B=gdf_p[gdf_p['Basin'] == gdf_p['Basin'][i]]
    lsm_R2.rio.set_spatial_dims(y_dim="lat",x_dim="lon", inplace=True)
    lsm_R2.rio.write_crs("EPSG:4326", inplace=True)
    lsm_clip = lsm_R2.rio.clip(B.geometry.apply(mapping), B.crs, drop=True)
    A=lsm_clip.mean(dim=['lat','lon']).expand_dims(dim='region')
    dataset.append(A)
    
lsm_R_ssn = xr.concat(dataset, dim='region')
lsm_R_ssn['region'] = np.array(gdf_p.Basin)
lsm_R_ssn.to_netcdf(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_reg_ssn.nc')

In [9]:
lsm_R_ssn.to_netcdf(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_reg_ssn.nc')

#### 3.2 Regional seasonal spi

In [11]:
import numpy as np
import xarray as xr
from scipy.stats import gamma, norm
#warnings.filterwarnings("ignore")

var=['TotalPrecip_tavg','Evap_tavg','Qs_tavg','RE','RE_cleaned']
# Define your function to calculate SPI
def calculate_spi(data_n):
    # Post-processing to handle NaNs and zeros

    # Replace NaN values with themean 
    data_n= np.where(np.isnan(data_n), np.nanmean(data_n), data_n)
    
    data_n=np.nan_to_num(data_n, nan=0)
    data_n = np.where((data_n == 0)|(data_n < 0), 0.01, data_n)  # Replace zeros with 0.01

    # Fit gamma distribution
    shape, loc, scale = gamma.fit(data_n, floc=0)

    # Calculate Probability of Exceedance (POE)
    POE = 1 - gamma.cdf(data_n, shape, loc, scale)

    # Calculate Standard Normal Variate (SNV) or Standardized Precipitation Index (SPI)
    cdf_std = norm.cdf(norm.ppf(POE))
    spi = norm.ppf(1 - cdf_std)

    return spi

# Assuming you have already loaded your dataset `lsm_ssn` which has dimensions (region, year, season)

lsm_R_ssn=lsm_R_ssn.chunk({'year': -1}) # Coree dimension should be planee 
lsm_reg_spi = xr.apply_ufunc(
    calculate_spi,  # Function for wavelet transformation
    lsm_R_ssn,  # Input dataset
    input_core_dims=[['year']],  # Core dimensions of the input
    output_core_dims=[['year']],  # Core dimensions of the output
    vectorize=True,  # Treat each lat-lon pair as a separate input
    dask='parallelized',  # Use parallelized computation if using Dask arrays
    output_dtypes=[lsm_R_ssn.RE.dtype],  # Output data type
    output_sizes={'year': len(lsm_R_ssn.year)}  # Size of the output dimension
)

lsm_reg_spi=lsm_reg_spi.where(~np.isinf(lsm_reg_spi),0)
lsm_reg_spi.to_netcdf(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_reg_ssn_SPI1.nc')


C:\Users\sstar\AppData\Local\Temp\ipykernel_30672\996908936.py:32: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  lsm_reg_spi = xr.apply_ufunc(


## 2 Regional average of RE (Here first we calculate average ppt and qs of a region then calculate RE)

In [187]:
gdf = gpd.read_file('DATA_p/Shapefiles/India_trans/India_trans_new.shp')

#merging east flowing
B1_merge = ['East_Flowing_M_G', 'East_Flowing_G_K', 'East_Flowing_K_P']
bas1 = gdf[gdf['Basin'].isin(B1_merge)]
m_geometry = bas1.geometry.unary_union;m_area = bas1.area.sum()
M1 = gpd.GeoDataFrame({'Basin': ['EFMP'], 'SUB_AREA': [m_area],'UP_AREA': [m_area], 'geometry': [m_geometry]}, crs=gdf.crs)

#merging east flowing
B2_merge = ['East_Flowing_Cauvery', 'East_flowing_pennar']
bas2 = gdf[gdf['Basin'].isin(B2_merge)]
m_geometry = bas2.geometry.unary_union;m_area = bas2.area.sum()
M2 = gpd.GeoDataFrame({'Basin': ['EFPK'], 'SUB_AREA': [m_area],'UP_AREA': [m_area], 'geometry': [m_geometry]}, crs=gdf.crs)

## APPENDING THEM
gdf_p = pd.concat([gdf, M1,M2], ignore_index=True)

## Dropingg the basins
B_drop=B1_merge+B2_merge+['east_flowing_mahdi_pnr','east_flowing_pnr_kanri','Brahmaputra','Ganga','Indus']
gdf_p=gdf_p[~gdf_p.Basin.isin(B_drop)]

gdf_p = gdf_p.reset_index(drop=True)
gdf_p.Basin[0:3]=['Brahmaputra','Ganga','Indus']
gdf_p.Basin[13]='West flowing'
gdf_p.Basin[14]='WFKS'
gdf_p.Basin

C:\Users\sstar\AppData\Local\Temp\ipykernel_11368\3816887693.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_new.Basin[0:3]=['Brahmaputra','Ganga','Indus']
C:\Users\sstar\AppData\Local\Temp\ipykernel_11368\3816887693.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_new.Basin[13]='West flowing'
C:\Users\sstar\AppData\Local\Temp\ipykernel_11368\3816887693.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_new.Basin

0      Brahmaputra
1            Ganga
2            Indus
3          Cauvery
4         Godavari
5          Krishna
6         Mahanadi
7             Mahi
8          Narmada
9           Pennar
10       Sabarmati
11    Subarnarekha
12            Tapi
13    West flowing
14            WFKS
15            EFMP
16            EFPK
Name: Basin, dtype: object

In [184]:
mdl=1
ssn=['Pre-monsoon','Monsoon','Post-monsoon','Winter']
ssn_m=[[3,4,5],[6,7,8,9],[10,11],[12,1,2]]
model=["LIS_MERRA2","LIS_IMD","LIS_CHIRPS","LIS_ERA5","LIS_IMDc"]
var=['TotalPrecip_tavg','Evap_tavg','Qs_tavg','RE']


lsm = xr.open_mfdataset(f"DATA_p/{model[mdl]}_1980_2021.nc",engine='netcdf4', parallel=True,chunks={'time': 'auto'})
lsm=lsm[['TotalPrecip_tavg','Evap_tavg','Qs_tavg']].compute()
lsm=lsm*60*60*24*30
lsm=lsm.where(lsm>=0)


dataset=[]
for i in range(len(basin_new)):
    
    B=gdf[gdf['Basin'] == basin_new[i]]
    lsm_R.rio.set_spatial_dims(y_dim="lat",x_dim="lon", inplace=True)
    lsm.rio.write_crs("EPSG:4326", inplace=True)
    lsm_clip = lsm.rio.clip(B.geometry.apply(mapping), B.crs, drop=True)
    A=lsm_clip.mean(dim=['lat','lon']).expand_dims(dim='region')
    dataset.append(A)
    
lsm_R = xr.concat(dataset, dim='region')
lsm_R['region'] = basin_new



### 2.1 Full timeseries

In [179]:
lsm_R1=lsm_R.where(lsm_R.TotalPrecip_tavg>=1) #isnull is to make the outer location nans remains nan
lsm_R1=lsm_R1.where(~(lsm_R1["Qs_tavg"]>lsm_R1["TotalPrecip_tavg"]))

RE=lsm_R1["Qs_tavg"]/lsm_R1["TotalPrecip_tavg"]
RE=RE.where(~np.isinf(RE)) #converting inf to nan (inf effect all operation like mean, trend etc)
RE=RE.where(RE<0.7).compute()
lsm_R['RE']=RE


RE_mn=RE.groupby("time.month").mean()
RE_mn = xr.concat([RE_mn]*len(np.unique(RE.time.dt.year)), dim="month")
RE_mn1=RE_mn.rename({'month': 'time'});RE_mn1["time"]=RE.time
RE_fin=RE.combine_first(RE_mn1)


lsm_R['RE_cleaned']=RE_fin

lsm_R.to_netcdf(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_reg_All.nc')


### 2.2 Full timeseries SPI

In [180]:
import numpy as np
import xarray as xr
from scipy.stats import gamma, norm
#warnings.filterwarnings("ignore")

var=['TotalPrecip_tavg','Evap_tavg','Qs_tavg','RE','RE_cleaned']
def calculate_spi(data):
    # Post-processing to handle NaNs and zeros
    data_n = data.copy()
    data_n = data_n.fillna(data_n.mean())  # Fill NaNs with mean
    data_n = xr.where(data_n == 0, 0.01, data_n)  # Replace zeros with 0.01

    # Fit gamma distribution
    shape, loc, scale = gamma.fit(data_n, floc=0)

    # Calculate Probability of Exceedance (POE)
    POE = 1 - gamma.cdf(data_n, shape, loc, scale)

    # Calculate Standard Normal Variate (SNV) or Standardized Precipitation Index (SPI)
    cdf_std = norm.cdf(norm.ppf(POE))
    spi = norm.ppf(1 - cdf_std)

    return spi


result=[]
for vr in var:

 ds=lsm_R[vr]
 spi = ds.groupby('time.month').apply(calculate_spi)

 # Create a new DataArray for SPI
 spi_ds = xr.DataArray(spi, coords=ds.coords, dims=ds.dims, name=f'SPI_{vr}')
 result.append(spi_ds)

spi_reg = xr.merge(result)
spi_reg.to_netcdf(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_reg_SPI1.nc')


### 2.3 Seasonal

In [185]:
lsm=lsm_R
dataset=[]
for ss in range(4):
    if ss!=3:
        lsm_ssn=lsm.sel(time=lsm["time.month"].isin(ssn_m[ss]))
        lsm_ssn=lsm_ssn.groupby("time.year").mean(dim="time")
        print("1")
    else:
        lsm_roll = lsm.rolling(time=3).mean()
        lsm_ssn=lsm_roll.isel(time=lsm_roll["time.month"].isin([1])).groupby("time.year").mean()
        lsm_ssn=lsm_ssn.where(lsm_ssn['year'] != 1980, lsm_ssn.isel(year=1))
        print("3")


    lsm_re=lsm_ssn.where(lsm_ssn.TotalPrecip_tavg>=0.4) #isnull is to make the outer location nans remains nan
    lsm_re1=lsm_re.where(~(lsm_ssn["Qs_tavg"]>lsm_ssn["TotalPrecip_tavg"]))
     
    RE=lsm_re1["Qs_tavg"]/lsm_re1["TotalPrecip_tavg"]
    RE=RE.where(~np.isinf(RE)) #converting inf to nan (inf effect all operation like mean, trend etc)
    RE=RE.where(RE<0.7).compute()
    lsm_ssn['RE']=RE
    
    #Preprocessing   
    RE_fin=RE.interpolate_na(dim="year", method="linear",fill_value="extrapolate") # lat or lon is also fine
    lsm_ssn['RE_cleaned']=RE_fin
    dataset.append(lsm_ssn.expand_dims(dim='season'))


lsm_ssn = xr.concat(dataset, dim='season')
lsm_ssn=lsm_ssn.assign_coords(season=ssn)


lsm_ssn.to_netcdf(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_reg_ssn.nc')



1
1
1
3


### 2.4 Seasonal SPI

In [186]:
import numpy as np
import xarray as xr
from scipy.stats import gamma, norm
#warnings.filterwarnings("ignore")

var=['TotalPrecip_tavg','Evap_tavg','Qs_tavg','RE','RE_cleaned']
# Define your function to calculate SPI
def calculate_spi(data_n):
    # Post-processing to handle NaNs and zeros

    # Replace NaN values with themean 
    data_n= np.where(np.isnan(data_n), np.nanmean(data_n), data_n)
    
    data_n=np.nan_to_num(data_n, nan=0)
    data_n = np.where((data_n == 0)|(data_n < 0), 0.01, data_n)  # Replace zeros with 0.01

    # Fit gamma distribution
    shape, loc, scale = gamma.fit(data_n, floc=0)

    # Calculate Probability of Exceedance (POE)
    POE = 1 - gamma.cdf(data_n, shape, loc, scale)

    # Calculate Standard Normal Variate (SNV) or Standardized Precipitation Index (SPI)
    cdf_std = norm.cdf(norm.ppf(POE))
    spi = norm.ppf(1 - cdf_std)

    return spi

# Assuming you have already loaded your dataset `lsm_ssn` which has dimensions (region, year, season)


lsm_reg_spi = xr.apply_ufunc(
    calculate_spi,  # Function for wavelet transformation
    lsm_ssn,  # Input dataset
    input_core_dims=[['year']],  # Core dimensions of the input
    output_core_dims=[['year']],  # Core dimensions of the output
    vectorize=True,  # Treat each lat-lon pair as a separate input
    dask='parallelized',  # Use parallelized computation if using Dask arrays
    output_dtypes=[ds.dtype],  # Output data type
    output_sizes={'time': len(lsm_ssn.year)}  # Size of the output dimension
)


lsm_reg_spi.to_netcdf(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_reg_ssn_SPI1.nc')


C:\Users\sstar\AppData\Local\Temp\ipykernel_11368\1301548028.py:32: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  lsm_reg_spi = xr.apply_ufunc(


# Practise session

In [138]:
import numpy as np
import xarray as xr
from scipy.stats import gamma, norm
#warnings.filterwarnings("ignore")

var=['TotalPrecip_tavg','Evap_tavg','Qs_tavg','RE','RE_cleaned']
def calculate_spi(data):
    # Post-processing to handle NaNs and zeros
    data_n = data.copy()
    data_n = data_n.fillna(data_n.mean())  # Fill NaNs with mean
    data_n = xr.where(data_n == 0, 0.01, data_n)  # Replace zeros with 0.01

    # Fit gamma distribution
    shape, loc, scale = gamma.fit(data_n, floc=0)

    # Calculate Probability of Exceedance (POE)
    POE = 1 - gamma.cdf(data_n, shape, loc, scale)

    # Calculate Standard Normal Variate (SNV) or Standardized Precipitation Index (SPI)
    cdf_std = norm.cdf(norm.ppf(POE))
    spi = norm.ppf(1 - cdf_std)

    return spi


result=[]
for vr in var:

 ds=lsm_R_all[vr]
 spi = ds.groupby('time.month').apply(calculate_spi)

 # Create a new DataArray for SPI
 spi_ds = xr.DataArray(spi, coords=ds.coords, dims=ds.dims, name=f'SPI_{vr}')
 result.append(spi_ds)

spi_reg = xr.merge(result)
spi_reg.to_netcdf(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_reg_SPI1.nc')


KeyboardInterrupt: 